# Encuesta Nacional de Nutrición y Salud (ENNyS2)

## Introducción

La Segunda Encuesta Nacional de Nutrición y Salud (ENNyS 2) es un relevamiento realizado en Argentina por la Secretaría de Gobierno de Salud de la Nación (hoy Ministerio de Salud) entre octubre de 2018 y febrero de 2019.

Su objetivo principal es describir la situación alimentaria y de salud de la población, recolectando información detallada sobre:

* Consumo de alimentos y bebidas (recordatorio de 24 horas, frecuencia y tipo de alimentos).

* Estado nutricional (mediciones antropométricas y datos de composición corporal).

* Enfermedades crónicas y factores de riesgo (hipertensión, diabetes, colesterol alto, etc.).

* Condiciones sociodemográficas y del hogar (cobertura de salud, nivel educativo, ingresos, etc.).

* Hábitos de vida (actividad física, lactancia, consumo de tabaco y alcohol, etc.).

La encuesta utiliza un diseño probabilístico, estratificado y multietápico, lo que permite que los resultados sean representativos a nivel nacional y por regiones.

### Se estructura en cuestionarios específicos por grupo etario:

* C1: Niños/as menores de 5 años.

* C2: Niños/as y adolescentes de 5 a 12 años.

* C3: Adolescentes de 13 a 17 años.

* C4: Adultos de 18 años o más.

En este trabajo utilizaremos la base principal publicada en el portal de datos abiertos del Ministerio de Salud, junto con las bases complementarias de alimentos y nutrientes, para realizar un análisis que vincule alimentación, estado de salud y factores asociados.

## Objetivos de la notebook

Explorar la base principal de la ENNyS2 (encuesta) para:
- Ubicar variables clave (edad, sexo, indicadores de salud, educación, región),
- Revisar cobertura de datos en adultos (≥18),
- Generar un subset mínimo limpio para notebooks posteriores.

## Estructura de las columnas en ENNyS2

La base principal de ENNyS2 contiene aproximadamente 1830 columnas.
Estas variables se organizan en módulos y prefijos que indican el tipo de información, la población objetivo y, en algunos casos, la posición del miembro del hogar.

### Convenciones de nombres:

Cn_: Variables de cuestionarios específicos según grupo etario:

C1_: Niños/as menores de 2 años (respondido por adulto responsable).

C2_: Niños/as y adolescentes de 2 a 12 años.

C3_: Adolescentes de 13 a 17 años.

C4_: Adultos de 18 años o más.

T_Cn_: Variables totales o derivadas de un módulo.
Ejemplo: T_C4_ActFisTotal (total de minutos de actividad física en adultos).

I_Dx_Cn_: Indicadores por día dentro del módulo.
Ejemplo: I_D1_C4_Kcal (ingesta calórica del día 1 en adultos).

Mn_: Información de miembros del hogar (hasta 15 posibles).
Ejemplo: M1_Edad, M2_Sexo (edad y sexo del miembro 1 y 2).

---

## 1. Cargar la base de personas y hogares

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

# Mostrar todas las filas de un DataFrame o Serie
pd.set_option("display.max_rows", None)

# Mostrar todas las columnas
pd.set_option("display.max_columns", None)

# Evitar el corte de anchos en los valores
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

DATA = Path("../data") # Previamente hay que descargar los datos manualmente y descomprimirlos en este directorio, mas detalles en el README.md
FILE = DATA / "ENNyS2_encuesta.csv"

df = pd.read_csv(FILE, encoding="utf-8-sig", low_memory=False) # Encoding para acentos

df.shape

(21358, 1830)

# Aclaración de cómo se realiza la encuesta

## 1. Formulario sociodemográfico (hogar e individuos)

* Todos los hogares seleccionados responden un formulario básico.

* Ahí se registran todos los integrantes: edad, sexo, parentesco, educación, variables socioeconómicas, etc.

* Esa es la información mínima, los llamados microdatos básicos.

## 2. Sorteo del individuo

* De cada hogar se elige un integrante al azar.

* Ese es el único que responde el cuestionario individual completo según su edad:

    * C1 → 0 a 23 meses

    * C2 → 2 a 12 años

    * C3 → 13 a 17 años

    * C4 → 18 años y más

**Importante**: solo el seleccionado tiene datos completos de salud, alimentación, antecedentes, etc.

## 3. Recordatorio de 24 horas (R24H)

* También se aplica solo al seleccionado.

* Registra todo lo consumido el día anterior (alimentos, bebidas, cantidades, horarios, contexto).

* En algunos casos se hace un segundo R24H en otro día, para captar variabilidad.

* Si no puede responder (bebés → se entrevista al adulto responsable, rechazos, etc.), el registro queda faltante.

* Estos datos se guardan en bases específicas:

    * Base de alimentos: cada fila es un alimento consumido.

    * Base de nutrientes: desglose de energía y nutrientes de esos alimentos.

## 4. Quienes no fueron sorteados

* El resto de los integrantes del hogar solo tienen los datos sociodemográficos básicos.

* No tienen cuestionario C1–C4 ni recordatorio de 24 hs.

## En la práctica de análisis

* Primero filtramos a los individuos con ES_Seleccionado = "Si", son los que efectivamente respondieron cuestionario individual + R24H. Son los de interes porque tienen informacion de alimentacion completa como para calcular el porcentaje de consumo de ultra procesados.

* Luego dividimos esos casos en los cuatro grupos etarios (C1, C2, C3, C4).

* Con eso trabajamos las variables de interés (ej. socioeconómicas, consumo, estado nutricional).

---

## 2. Separamos en 4 subbases (C1 a C4)


In [2]:
df_sel = df[df["ES_Seleccionado"]=="Si"]

c1 = df_sel[df_sel["E_CUEST"]=="0 a 23 meses"]
c2 = df_sel[df_sel["E_CUEST"]=="2 a 12 años"]
c3 = df_sel[df_sel["E_CUEST"]=="13 a 17 años"]
c4 = df_sel[df_sel["E_CUEST"]=="18 o mas años"]

print("0 a 23 meses:", c1.shape[0])
print("2 a 12 años:", c2.shape[0])
print("13 a 17 años:", c3.shape[0])
print("18 años y más:", c4.shape[0])

0 a 23 meses: 2013
2 a 12 años: 216
13 a 17 años: 401
18 años y más: 2876


---
## Examinamos las columnas

In [3]:
# 1) Limpieza de "falsos vacíos"
NULL_TOKENS = ["", " ", "No aplica edad", "No aplica pm", "Sin clave", "NS/NR", "No sabe", "No contesta"]

def normalizar_nulos(df):
    return df.replace(NULL_TOKENS, np.nan)

# 2) Ranking  de completitud (sin filtrar columnas)
def ranking_completitud_simple(df, nombre, top=200):
    df2 = normalizar_nulos(df)
    comp = df2.notna().mean()  # proporción de no nulos por columna
    comp = comp.sort_values(ascending=False)

    print(f"\n[{nombre}] {df2.shape[0]} registros | {df2.shape[1]} columnas (todas)")
    print(f"\nTop {top} columnas con más datos:")
    print(comp.head(top))

    return comp

# 3) Ejecutar para cada subbase (usa TODAS las columnas)
comp_c1 = ranking_completitud_simple(c1, "C1 (0-23 meses)")
comp_c2 = ranking_completitud_simple(c2, "C2 (2-12 años)")
comp_c3 = ranking_completitud_simple(c3, "C3 (13-17 años)")
comp_c4 = ranking_completitud_simple(c4, "C4 (18+ años)")


/tmp/ipykernel_24448/3985329528.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.replace(NULL_TOKENS, np.nan)



[C1 (0-23 meses)] 2013 registros | 1830 columnas (todas)

Top 200 columnas con más datos:
C1_SP_3                           1.000000
activo_OMS_adol                   1.000000
id                                1.000000
Srvyr                             1.000000
M01_sd_caso_id                    1.000000
M01_SD_3                          1.000000
M01_SD_4                          1.000000
ES_Seleccionado                   1.000000
miembro_id                        1.000000
hogar_id                          1.000000
VStart                            1.000000
M01_SD_6                          1.000000
M01_SD_10                         1.000000
M01_SD_12                         1.000000
M01_SD_14_O1                      1.000000
vivienda_id                       1.000000
region                            1.000000
M01_SD_15                         1.000000
SD_28                             1.000000
M01_SD_6_1                        1.000000
M01_SD_6_2                        1.000000
predic

---

## 3. Cargamos base de alimentos

In [4]:
FILE = DATA / "Base_Alimentos_Bebidas_Suplementos.csv"

alimentos = pd.read_csv(FILE, encoding="utf-8-sig", low_memory=False) # Encoding para acentos

alimentos.shape

(26028, 1114)

---

## 3. Seleccionamos columnas relevantes

In [5]:
# === 2. Columnas relevantes ===
columnas_relevantes = [
    # Datos básicos
    "Edadd", "C4_SEXO", "IMC", "Cobertura_salud", "IngHog_UC_QUINT_imp", "region",
    
    # Enfermedades crónicas
    "T_C4_SG_1_2_1",  # Diabetes
    "T_C4_SG_1_2_2",  # Enfermedad renal
    "T_C4_SG_1_2_3",  # Hipertensión o presión alta
    "T_C4_SG_1_2_4",  # Dislipemia, colesterol o triglicéridos altos
    "T_C4_SG_1_2_5",  # Anemia
    "T_C4_SG_1_2_6",  # Cardiopatía
    "T_C4_SG_1_2_7",  # Sonda vesical
    "T_C4_SG_1_2_8",  # Transplante de riñón
    "T_C4_SG_1_2_9",  # Obesidad
    "T_C4_SG_1_2_10", # EPOC
    "T_C4_SG_1_2_11", # Infarto
    "T_C4_SG_1_2_12", # Cáncer
    "T_C4_SG_1_2_13", # Leucemia
    "T_C4_SG_1_2_14"  # Otras enfermedades de larga duración
]

# === 3. Renombrar columnas ===
nombres_amigables = {
    "Edadd": "edad",
    "C4_SEXO": "sexo",
    "IMC": "imc",
    "Cobertura_salud": "cobertura_salud",
    "IngHog_UC_QUINT_imp": "quintil_ingresos",
    "region": "region",
    "T_C4_SG_1_2_1": "diabetes",
    "T_C4_SG_1_2_2": "enf_renal",
    "T_C4_SG_1_2_3": "hipertension",
    "T_C4_SG_1_2_4": "dislipemia",
    "T_C4_SG_1_2_5": "anemia",
    "T_C4_SG_1_2_6": "cardiopatia",
    "T_C4_SG_1_2_7": "sonda_vesical",
    "T_C4_SG_1_2_8": "transplante_renal",
    "T_C4_SG_1_2_9": "obesidad",
    "T_C4_SG_1_2_10": "epoc",
    "T_C4_SG_1_2_11": "infarto",
    "T_C4_SG_1_2_12": "cancer",
    "T_C4_SG_1_2_13": "leucemia",
    "T_C4_SG_1_2_14": "otras_enf_larga_duracion"
}

df_sel = df_ad[columnas_relevantes].rename(columns=nombres_amigables)

df_sel.head()


NameError: name 'df_ad' is not defined